In [1]:
import add_packages
import config
from pprint import pprint

from my_langchain import (
    document_loaders, text_splitters, text_embedding_models, vector_stores,
    chat_models, prompts, utils, output_parsers, agents, documents, runnables,
    llms, histories, tools
)

# Quickstart

Build an agent with two tools: one for online searches and one for specific data retrieval from an index.

## Define tools

In [2]:
# Create the tools needed: Tavily for online search and a retriever for local index.

tool_tavily_search = tools.tavily_search_results()

# Create a retriever over data. 
loader = document_loaders.WebBaseLoader("https://docs.smith.langchain.com/")
document = loader.load()
documents = text_splitters.RecursiveCharacterTextSplitter(
  chunk_size=1000, chunk_overlap=200,
).split_documents(document)

embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = vector_stores.chroma.Chroma.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()
tool_retriever = vector_stores.create_retriever_tool(
  retriever=retriever,
  name="langsmith_search",
  description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

# List of tools will use downstream.
tools = [
  tool_tavily_search, 
  tool_retriever,
]

## Create agent

In [3]:

# Choose LLM guiding agent.
llm = chat_models.chat_openai

# Choose the prompt to guide the agent.
prompt = prompts.hub.pull("hwchase17/openai-functions-agent")

# Initialize the agent with the LLM, prompt, and tools. 
# The agent takes in input and decides on actions. 
# AgentExecutor execute actions for Agent
agent = agents.create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

## Run agent

In [4]:

# Run agent on stateless queries.
agent_executor.invoke({"input": "hi"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi', 'output': 'Hello! How can I assist you today?'}

## Adding in memory

This agent is stateless, does not remember previous interactions. To give it memory, pass in previous chat_history. It needs to be called chat_history because of the prompt used. If a different prompt is used, the variable name could be changed.

Keep track of messages automatically by wrapping in a RunnableWithMessageHistory. 

In [9]:
# Chat history is stored in memory using a global Python dictionary.
store = {}

def get_session_history(
  user_id: str, conversation_id: str
) -> histories.BaseChatMessageHistory:
  """
  Callable references a dict to return an instance of ChatMessageHistory. 
  
  The arguments can be specified by passing a configuration to the 
  RunnableWithMessageHistory at runtime. 
  
  The configuration parameters for tracking message histories can be customized 
  by passing a list of ConfigurableFieldSpec objects to the 
  history_factory_config parameter. 
  
  Two parameters used are user_id and conversation_id.
  """
  if (user_id, conversation_id) not in store:
    store[(user_id, conversation_id)] = histories.ChatMessageHistory()
  return store[(user_id, conversation_id)]

agent_with_memory = runnables.RunnableWithMessageHistory(
  agent_executor,
  get_session_history,
  input_messages_key="input",  # latest input message
  history_messages_key="history",  # key to add historical messages to
  history_factory_config=[
    runnables.ConfigurableFieldSpec(
      id="user_id", annotation=str, name="User ID", default="",
      description="Unique identifier for the user.", is_shared=True,
    ),
    runnables.ConfigurableFieldSpec(
      id="conversation_id", annotation=str, name="Conversation ID", default="", 
      description="Unique identifier for the conversation.", is_shared=True,
    ),
  ]
)

print(agent_with_memory.invoke(
    {"input": "Hi, I'm Bob"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}}
))

print(agent_with_memory.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}}
))



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.
{'input': "Hi, I'm Bob", 'history': [], 'output': 'Hello Bob! How can I assist you today?'}


> Entering new AgentExecutor chain...
I'm sorry, but I don't have access to personal information like your name. How can I assist you today?

> Finished chain.
{'input': 'What is my name?', 'history': [HumanMessage(content="Hi, I'm Bob"), AIMessage(content='Hello Bob! How can I assist you today?')], 'output': "I'm sorry, but I don't have access to personal information like your name. How can I assist you today?"}


In [13]:
message_history = histories.ChatMessageHistory()

agent_with_chat_history = runnables.RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


> Entering new AgentExecutor chain...
Your name is Bob! How can I assist you today, Bob?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob! How can I assist you today, Bob?'}

# Concepts


# Agent Types


### OpenAI tools


### XML Agent


### JSON Chat Agent


### Structured chat


### ReAct


### Self-ask with search

# How-to


### Custom agent


### Streaming


### Structured Tools


### Running Agent as an Iterator


### Returning Structured Output


### Handle parsing errors


### Access intermediate steps


### Cap the max number of iterations


### Timeouts for agents

# Tools

### Toolkits


### Defining Custom Tools


### Tools as OpenAI Functions